In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier


In [5]:
df = pd.read_csv("preprocessed.csv")
df.drop(["id","comment_text"],axis=1,inplace=True)
df.head()


,toxic,severe_toxic,obscene,threat,insult,identity_hate,non_toxic,prepocessed_comment
0,1,1,1,0,1,0,0,COCKSUCKER pis work
1,1,0,0,0,0,0,0,hey | talk .What exclusive group WP taliban go...
2,1,0,0,0,0,0,0,Bye look come think comme Tosser
3,1,0,1,0,1,1,0,gay antisemmitian Archangel WHite TigerMeow Gr...
4,1,0,1,0,1,0,0,fuck filthy mother ASS DRY


## check na

In [6]:
df.isnull().sum()

toxic                   0
severe_toxic            0
obscene                 0
threat                  0
insult                  0
identity_hate           0
non_toxic               0
prepocessed_comment    29
dtype: int64

In [8]:
df.shape

(64900, 8)

In [26]:
cols=[col for col in df.columns]
for c in cols:
    if c!="prepocessed_comment":
        print(df[c].value_counts())

toxic
0    49606
1    15294
Name: count, dtype: int64
severe_toxic
0    63305
1     1595
Name: count, dtype: int64
obscene
0    56451
1     8449
Name: count, dtype: int64
threat
0    64422
1      478
Name: count, dtype: int64
insult
0    57023
1     7877
Name: count, dtype: int64
identity_hate
0    63495
1     1405
Name: count, dtype: int64
non_toxic
1    48675
0    16225
Name: count, dtype: int64


HIGHLY IMBALANCED classes but still toxic  message class can be off but it will still classified as toxic ,get removed so purpose is solved majorly 


1)toxic or non-toxic (simple,highly accurate)


2)multiclass(complex,inaccurate)  THIS WAY IS TAKEN

In [32]:
df.dropna(inplace=True)

In [33]:
df.isnull().sum()

toxic                  0
severe_toxic           0
obscene                0
threat                 0
insult                 0
identity_hate          0
non_toxic              0
prepocessed_comment    0
dtype: int64

In [36]:
#lets see classtype

for col in cols:
    types=df[col].map(type).unique()
    print(f"Cols=>{col}",types)

Cols=>toxic [<class 'int'>]
Cols=>severe_toxic [<class 'int'>]
Cols=>obscene [<class 'int'>]
Cols=>threat [<class 'int'>]
Cols=>insult [<class 'int'>]
Cols=>identity_hate [<class 'int'>]
Cols=>non_toxic [<class 'int'>]
Cols=>prepocessed_comment [<class 'str'>]


In [37]:
# so data types are ok

In [38]:
X=df['prepocessed_comment']
Y=df.drop('prepocessed_comment',axis=1)

In [39]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [41]:
print(f"Shape x_train{x_train.shape} ,x_test{x_test.shape} ,y_train{y_train.shape} ,t_test{y_test.shape}")

Shape x_train(51896,) ,x_test(12975,) ,y_train(51896, 7) ,t_test(12975, 7)


In [42]:
vectorizer = TfidfVectorizer(max_features=50000)
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec  = vectorizer.transform(x_test)

## Lr model One-vs-rest Classifier 
my reason is be simple in model(Logistic regression ) so extend it to multiclass

In [43]:
model = OneVsRestClassifier(
    LogisticRegression(max_iter=2000)
)
model.fit(x_train_vec, y_train)


OneVsRestClassifier(estimator=LogisticRegression(max_iter=2000))

In [44]:
y_pred = model.predict(x_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.71      0.79      3057
           1       0.56      0.21      0.31       315
           2       0.89      0.66      0.76      1668
           3       0.64      0.14      0.23        99
           4       0.79      0.53      0.63      1572
           5       0.69      0.19      0.29       280
           6       0.92      0.98      0.95      9742

   micro avg       0.90      0.82      0.86     16733
   macro avg       0.77      0.49      0.57     16733
weighted avg       0.89      0.82      0.84     16733
 samples avg       0.89      0.87      0.88     16733



C:\Users\BIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [45]:
label_names = [
    "toxic", "severe_toxic", "obscene",
    "threat", "insult", "identity_hate"
]

def predict_label(text):
    vec = vectorizer.transform([text])
    pred = model.predict(vec)[0]      # array like [0 1 0 0 0 0]

    if pred.sum() == 0:
        return "non_toxic"

    # if multiple labels triggered → return all
    output = [label_names[i] for i,v in enumerate(pred) if v==1]
    return ", ".join(output)


In [46]:
test_sentence = "you brown people are dumb"
print(predict_label(test_sentence))

toxic, insult


In [48]:
import joblib

model_path = "moderator_model.pkl"

joblib.dump({
    "vectorizer": vectorizer,
    "model": model
}, model_path)

print("Saved:", model_path)

Saved: moderator_model.pkl


In [ ]:
df[]